<h2>PREPARING PIPELINE TO REMOVE DUPLICATE READS</h2>
<p>This pipeline maps the reads to the complete genome of B. dolosa AU0158 using BWA v. 0.7.17<sup>1</sup>. The resulting mapping results are sorted with Samtools v. 1.9<sup>2</sup>. Then, the MarkDuplicates and SamToFastq functions of picard v. 2.18.4<sup>3</sup> are employed to remove duplicate reads and to convert the results back to fastq, respectively.</p>
<p>The results are located at:
<ul>
<li>Deduplicated reads can be found in <b>results/dedupper/&lt;ISOLATE_ID&gt;_1.ddp.fq</b> AND <b>results/dedupper/&lt;ISOLATE_ID&gt;_2.ddp.fq</b></li>
<li>Summary of reads removed can be found in <b>results/dedupper/DuplicateRemovalSummary.tsv</b></li>
</ul>
</p>

In [ ]:
cd results/dedupper

In [ ]:
#Setup variables for the deduplication pipeline
export BWA=../../bin/bwa-0.7.17/bwa
export SAMTOOLS=../../bin/samtools-1.9/samtools
export PICARD=../../bin/picard.jar

In [ ]:
#Index reference
../../bin/bwa-0.7.17/bwa index ../../data/reference/au0158/au0158.fa

In [ ]:
#Write the pipeline to remove duplicate reads for each isolate. This requires python 3
for i in `cat ../../doc/IdList.txt`; do ../../bin/fq_dedupper.py -o1 $i"_1.ddp.fq" -o2 $i"_2.ddp.fq" -b -ob $i".sh" ../../data/reference/au0158/au0158.fa "../../data/wgs/"$i"_1.fq" "../../data/wgs/"$i"_2.fq"; done

<h2>EXECUTING THE PIPELINE</h2>
<h3>SUBMITTING JOBS TO SCINET</h3>

In [ ]:
#Create job for scinet including isolates 1a-4j
echo -e '#!/bin/bash'"\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=40\n#SBATCH --time=5:00:00\n#SBATCH --job-name=dedupper_jobs-A\n#SBATCH --output=dedupper_jobs-A\nmodule load java" > dedupperJob_A.sh
for i in `head -40 ../../doc/IdList.txt`; do echo "bash "$i".sh 2> "$i".err >"$i".out &" >> dedupperJob_A.sh; done
echo "wait" >> dedupperJob_A.sh

In [ ]:
#Create job for scinet including isolates 5a-8j
echo -e '#!/bin/bash'"\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=40\n#SBATCH --time=5:00:00\n#SBATCH --job-name=dedupper_jobs-B\n#SBATCH --output=dedupper_jobs-B\nmodule load java" > dedupperJob_B.sh
for i in `tail -40 ../../doc/IdList.txt`; do echo "bash "$i".sh 2> "$i".err >"$i".out &" >> dedupperJob_B.sh; done
echo "wait" >> dedupperJob_B.sh

In [ ]:
#Submit jobs to scinet
sbatch dedupperJob_A.sh
sbatch dedupperJob_B.sh

<h3>OR RUNNING PIPELINE LOCALLY (THIS WILL TAKE A LONG TIME)</h3>

In [ ]:
for i in `cat ../../doc/IdList.txt`; do bash $i".sh"; done

<h2>SUMMARIZE </h2>
<p>Record the number of reads and bases left over after trimmin duplicate reads.</p>

In [ ]:
echo -e "IsolateId\tReadOneCount\tReadTwoCount\tBaseCountOne\tBaseCountTwo" > DuplicateRemovalSummary.tsv 
for i in `cat ../../doc/IdList.txt`; do RN_1=`awk -v I=$i 'NR%4==1' $i"_1.ddp.fq" | wc -l`;RN_2=`awk -v I=$i 'NR%4==1' $i"_2.ddp.fq" | wc -l`;BC_1=`awk -v I=$i 'NR%4==2 {printf $1}' $i"_1.ddp.fq" | wc -m`;BC_2=`awk -v I=$i 'NR%4==2 {printf $1}' $i"_2.ddp.fq"| wc -m`; echo -e $i"\t"$RN_1"\t"$RN_2"\t"$BC_1"\t"$BC_2; done >> DuplicateRemovalSummary.tsv

<sup>1</sup> http://bio-bwa.sourceforge.net
<br><sup>2</sup> https://www.htslib.org
<br><sup>3</sup> https://github.com/broadinstitute/picard/releases/tag/2.18.4